In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 3.1/3.1 GB ? eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 40.1 MB/s eta 0:00:00
     ------------------------------------ 109.3/109.3 kB 576.0 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1
    Uninstalling torchvision-0.13.1:
      Successfully uninstalled torchvision-0.13.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1
    Uninstalling torchaudio-0.12.1:
      Successfully uninstalled torchaudio-0.12.1


In [2]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [4]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
result = model(tokens)
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [5]:
torch.max(result.logits).item()

1.9803773164749146

In [6]:
type(torch.max(result.logits).item())

float

In [7]:
torch.max(result.logits[0])

tensor(1.9804, grad_fn=<MaxBackward1>)

In [8]:
df = pd.read_csv('sentiment.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     3534 non-null   object
 1   text       3534 non-null   object
 2   sentiment  3534 non-null   object
dtypes: object(3)
memory usage: 83.0+ KB


In [10]:
df.isnull().sum()

textID       0
text         0
sentiment    0
dtype: int64

In [11]:
def sentiment_score(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    return torch.max(result.logits).item()

In [12]:
df['bert_sentiment'] = df['text'].apply(sentiment_score)

In [13]:
df.head()

,textID,text,sentiment,bert_sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,0.310485
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive,1.160776
2,eee518ae67,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!",negative,2.975873
3,01082688c6,happy bday!,positive,2.256341
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,1.900328


In [14]:
df['text_len'] = df['text'].str.split().apply(len)

In [15]:
df.head()

,textID,text,sentiment,bert_sentiment,text_len
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,0.310485,6
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive,1.160776,15
2,eee518ae67,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!",negative,2.975873,13
3,01082688c6,happy bday!,positive,2.256341,2
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,1.900328,5
